In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida
from models.polishing import polishing
from models.filtering import filtering

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_miocardial()
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=.1)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (1700, 110)
5.62% missing data
Class distribution: (array([0, 1]), array([1429,  271]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': {
        'input_channels':1,
        'output_channels':1,
        'channels_skip':4,
        'down_layers':[(24, 7, 2),
                       (46, 9, 1),
                       (96, 9, 5),
                       (96, 11, 1)],
        'need_sigmoid':True,
        'need_bias':True,
        'pad':'zero',
        'downsample_mode':'stride',
        'upsample_mode':'nearest',
        'act_fun':'LeakyReLU',
        'need1x1_up':True
    },# or a list containing layers sizes
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 501, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 2.54 sec - Loss 0.223435 - ACC 74.51% - ACC Mean 74.51% - AUC 80.73% - AUC Mean 80.73% - Deter 000
Ite 00050 - 0.97 sec - Loss 0.108350 - ACC 72.74% - ACC Mean 73.83% - AUC 79.37% - AUC Mean 79.82% - Deter 042
Ite 00058 - 0.93 sec - Loss 0.102232 - ACC 71.96% - ACC Mean 73.49% - AUC 78.38% - AUC Mean 79.50% - Deter 050
Early stop ite 58, rollback to correction of ite 8, whith acc of 75.7% and auc of 82.71%
OUR METHOD RUN 1/10 - acc: 75.7% +- 0.0% - auc: 82.71% +- 0.0%
Ite 00000 - 0.99 sec - Loss 0.231320 - ACC 75.43% - ACC Mean 75.43% - AUC 80.16% - AUC Mean 80.16% - Deter 000
Ite 00050 - 0.91 sec - Loss 0.121362 - ACC 73.89% - ACC Mean 74.40% - AUC 80.55% - AUC Mean 80.49% - Deter 049
Ite 00051 - 0.96 sec - Loss 0.120488 - ACC 73.71% - ACC Mean 74.36% - AUC 80.60% - AUC Mean 80.44% - Deter 050
Early stop ite 51, rollback to correction of ite 1, whith acc of 75.44% and auc of 82.68%
OUR METHOD RUN 2/10 - acc: 75.44% +- 0.0% - auc: 82.68% +- 0.0%
Ite 00000 - 0.81 sec - Loss 

## Run MICE

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 73.1% +- 0.0% - auc: 78.16% +- 0.0%


## Run SFIL

In [6]:
sfil_accs, sfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='standard', random_state=i)
    sfil_accs.append(acc.mean())
    sfil_aucs.append(auc.mean())
    print(f'SFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SFIL GLOBAL - acc: {round(np.mean(sfil_accs) * 100, 2)}% +- {round(np.std(sfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(sfil_aucs) * 100, 2)}% +- {round(np.std(sfil_aucs) * 100, 2)}%')

SFIL RUN 1/10 - acc: 56.16% +- 2.22% - auc: 62.49% +- 5.5%
SFIL RUN 2/10 - acc: 56.91% +- 3.86% - auc: 63.8% +- 6.53%
SFIL RUN 3/10 - acc: 55.66% +- 1.27% - auc: 57.73% +- 1.79%
SFIL RUN 4/10 - acc: 56.85% +- 2.36% - auc: 58.69% +- 1.73%
SFIL RUN 5/10 - acc: 56.83% +- 0.98% - auc: 58.66% +- 2.81%
SFIL RUN 6/10 - acc: 56.88% +- 1.79% - auc: 62.35% +- 2.39%
SFIL RUN 7/10 - acc: 56.91% +- 2.28% - auc: 59.8% +- 2.98%
SFIL RUN 8/10 - acc: 56.83% +- 0.78% - auc: 58.92% +- 2.94%
SFIL RUN 9/10 - acc: 56.9% +- 2.25% - auc: 58.42% +- 4.08%
SFIL RUN 10/10 - acc: 56.9% +- 1.7% - auc: 60.33% +- 1.77%
SFIL GLOBAL - acc: 56.68% +- 0.4% - auc: 60.12% +- 1.96%


## Run PFIL

In [7]:
pfil_accs, pfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='panda', random_state=i)
    pfil_accs.append(acc.mean())
    pfil_aucs.append(auc.mean())
    print(f'PFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PFIL GLOBAL - acc: {round(np.mean(pfil_accs) * 100, 2)}% +- {round(np.std(pfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(pfil_aucs) * 100, 2)}% +- {round(np.std(pfil_aucs) * 100, 2)}%')

PFIL RUN 1/10 - acc: 70.31% +- 3.46% - auc: 76.91% +- 6.61%
PFIL RUN 2/10 - acc: 71.1% +- 2.63% - auc: 75.16% +- 2.56%
PFIL RUN 3/10 - acc: 71.84% +- 3.25% - auc: 76.3% +- 2.24%
PFIL RUN 4/10 - acc: 70.69% +- 4.45% - auc: 78.12% +- 2.39%
PFIL RUN 5/10 - acc: 72.56% +- 3.03% - auc: 75.47% +- 5.63%
PFIL RUN 6/10 - acc: 71.95% +- 3.42% - auc: 76.66% +- 5.14%
PFIL RUN 7/10 - acc: 71.93% +- 3.4% - auc: 78.15% +- 3.81%
PFIL RUN 8/10 - acc: 71.64% +- 2.8% - auc: 77.51% +- 3.1%
PFIL RUN 9/10 - acc: 72.84% +- 2.56% - auc: 77.8% +- 2.69%
PFIL RUN 10/10 - acc: 71.44% +- 2.95% - auc: 76.43% +- 4.02%
PFIL GLOBAL - acc: 71.63% +- 0.74% - auc: 76.85% +- 1.0%


## Run SPOL

In [8]:
spol_accs, spol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='standard', random_state=i)
    spol_accs.append(acc.mean())
    spol_aucs.append(auc.mean())
    print(f'SPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SPOL GLOBAL - acc: {round(np.mean(spol_accs) * 100, 2)}% +- {round(np.std(spol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(spol_aucs) * 100, 2)}% +- {round(np.std(spol_aucs) * 100, 2)}%')

SPOL RUN 1/10 - acc: 60.21% +- 6.18% - auc: 64.19% +- 5.65%
SPOL RUN 2/10 - acc: 57.96% +- 4.75% - auc: 64.93% +- 7.17%
SPOL RUN 3/10 - acc: 58.76% +- 3.18% - auc: 62.56% +- 3.6%
SPOL RUN 4/10 - acc: 55.14% +- 1.72% - auc: 62.01% +- 5.27%
SPOL RUN 5/10 - acc: 58.69% +- 1.79% - auc: 66.08% +- 2.54%
SPOL RUN 6/10 - acc: 60.14% +- 5.95% - auc: 63.29% +- 7.32%
SPOL RUN 7/10 - acc: 55.78% +- 1.33% - auc: 61.75% +- 1.8%
SPOL RUN 8/10 - acc: 60.14% +- 3.9% - auc: 64.19% +- 2.89%
SPOL RUN 9/10 - acc: 62.56% +- 6.17% - auc: 64.61% +- 5.68%
SPOL RUN 10/10 - acc: 62.57% +- 5.74% - auc: 67.28% +- 6.82%
SPOL GLOBAL - acc: 59.2% +- 2.36% - auc: 64.09% +- 1.67%


## Run PPOL

In [9]:
ppol_accs, ppol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='panda', random_state=i)
    ppol_accs.append(acc.mean())
    ppol_aucs.append(auc.mean())
    print(f'PPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PPOL GLOBAL - acc: {round(np.mean(ppol_accs) * 100, 2)}% +- {round(np.std(ppol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(ppol_aucs) * 100, 2)}% +- {round(np.std(ppol_aucs) * 100, 2)}%')

PPOL RUN 1/10 - acc: 72.47% +- 3.12% - auc: 77.03% +- 2.28%
PPOL RUN 2/10 - acc: 70.78% +- 2.83% - auc: 74.23% +- 2.63%
PPOL RUN 3/10 - acc: 70.29% +- 4.12% - auc: 75.99% +- 3.45%
PPOL RUN 4/10 - acc: 71.8% +- 2.69% - auc: 78.97% +- 4.02%
PPOL RUN 5/10 - acc: 71.04% +- 2.79% - auc: 76.88% +- 4.04%
PPOL RUN 6/10 - acc: 71.62% +- 2.37% - auc: 76.09% +- 2.06%
PPOL RUN 7/10 - acc: 72.1% +- 3.48% - auc: 77.8% +- 4.82%
PPOL RUN 8/10 - acc: 71.76% +- 2.26% - auc: 78.39% +- 2.69%
PPOL RUN 9/10 - acc: 72.62% +- 2.08% - auc: 78.34% +- 2.05%
PPOL RUN 10/10 - acc: 72.95% +- 1.65% - auc: 78.39% +- 3.71%
PPOL GLOBAL - acc: 71.74% +- 0.8% - auc: 77.21% +- 1.39%


## Run T-tests

In [10]:
for model, metrics in {
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'SFIL': {'ACC': sfil_accs, 'AUC': sfil_aucs},
        'PFIL': {'ACC': pfil_accs, 'AUC': pfil_aucs},
        'SPOL': {'ACC': spol_accs, 'AUC': spol_aucs},
        'PPOL': {'ACC': ppol_accs, 'AUC': ppol_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly better than MICE
Metric AUC - OUR METHOD is significantly better than MICE
Metric ACC - OUR METHOD is significantly better than SFIL
Metric AUC - OUR METHOD is significantly better than SFIL
Metric ACC - OUR METHOD is significantly better than PFIL
Metric AUC - OUR METHOD is significantly better than PFIL
Metric ACC - OUR METHOD is significantly better than SPOL
Metric AUC - OUR METHOD is significantly better than SPOL
Metric ACC - OUR METHOD is significantly better than PPOL
Metric AUC - OUR METHOD is significantly better than PPOL
